In [42]:
# Imports
import psycopg2
import pandas as pd

In [43]:
# Returns user database
def get_users(hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # Establish the connection
    conn = psycopg2.connect(host=hostname, port=port, user=postgre_user, dbname='users')

    # Query to select all users
    query = "SELECT * FROM users;"

    # Read query result directly into pandas DataFrame
    df = pd.read_sql_query(query, conn)

    # Close the connection and return
    conn.close()
    return df

In [44]:
def add_user(username, email, password_hash, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # Establish the connection
    conn = psycopg2.connect(host=hostname, port=port, user=postgre_user, dbname='users')

    # SQL query to insert a new user
    query = f"INSERT INTO users (username, email, password_hash) VALUES ('{username}', '{email}', '{password_hash}');"
    
    # Create cursor to execute query and commit changes
    cursor = conn.cursor()
    cursor.execute(query, (username, email)) # Assert user and email is unique
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

In [45]:
def remove_user(username, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # Establish the connection
    conn = psycopg2.connect(host=hostname, port=port, user=postgre_user, dbname='users')

    # SQL query to delete a user based on the username
    query = f"DELETE FROM users WHERE username = '{username}';"
    
    # Create cursor to execute query and commit changes
    cursor = conn.cursor()
    cursor.execute(query, (username)) # Assert user is unique
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

In [46]:
# Get DB example
df = get_users()
df

/var/folders/r3/6v_h8z6936vbpxsh4znh6mbm0000gn/T/ipykernel_59778/1206838723.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,username,email,password_hash
0,john_doe,john@example.com,hashed_password_123
1,jane_doe,jane@example.com,hashed_password_456


In [47]:
# Add user example
remove_user("chungus") # Ensure doesn't exist since we're asserting users are unique
add_user("chungus", "bingus@yahoo.co.in", "hashed_password_222")
df = get_users()
df

/var/folders/r3/6v_h8z6936vbpxsh4znh6mbm0000gn/T/ipykernel_59778/1206838723.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,username,email,password_hash
0,john_doe,john@example.com,hashed_password_123
1,jane_doe,jane@example.com,hashed_password_456
2,chungus,bingus@yahoo.co.in,hashed_password_222


In [48]:
# Remove user example
remove_user("chungus")
df = get_users()
df

/var/folders/r3/6v_h8z6936vbpxsh4znh6mbm0000gn/T/ipykernel_59778/1206838723.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,username,email,password_hash
0,john_doe,john@example.com,hashed_password_123
1,jane_doe,jane@example.com,hashed_password_456
